In [1]:
#get transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#get large GPT2 tokenizer and GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

/home/bk123477/decs_jupyter_lab/bartvenv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import json
import torch
from torch.utils.data import Dataset, DataLoader

import numpy as np
from tqdm import tqdm
import re
from pprint import pprint

In [3]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5" # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# train_lists = []
# label_lists = []
# with open("test_session_2.jsonl") as f:
#     for json_line in f:
#         current_strs = ""
#         prev_strs = ""
#         json_file = json.loads(json_line)
#         current_conversation = json_file["dialog"]
#         for item1 in current_conversation:
#             current_strs += f"{item1['text']} " 
#         train_lists.append(current_strs)
#         prev_conversation = json_file["previous_dialogs"]
#         previous_dialogue =[item['dialog'] for item in prev_conversation]
#         for item2 in previous_dialogue[0]:
#             prev_strs += f"{item2['text']} "
#         train_lists.append(prev_strs)
        
# lengdata = len(train_lists)

train_lists = []
label_lists = []
with open("test_session_2.jsonl") as f:
    for json_line in f:
        current_strs = ""
        prev_strs = ""
        json_file = json.loads(json_line)
        current_conversation = json_file["dialog"]
        for i, dic in enumerate(current_conversation):
            if i == 0:
                current_strs = f"Speaker {i%2 + 1} : "+ dic["text"] + " "               
            if i != 0 and i != len(current_conversation)-1:
                train_lists.append(current_strs)
                current_strs += f"Speaker {i%2 + 1} : "+ dic["text"] + " "
                label_lists.append(dic["text"])
            if i == len(current_conversation)-1:
                train_lists.append(current_strs)
                label_lists.append(dic["text"])
                
lengdata = len(train_lists)
print(train_lists[0])
print(label_lists[0])
print(train_lists[1])

In [5]:
file_name2 = "bart_all_session_no_persona/session_2/train.jsonl"

train_lists = []
label_lists = []

with open(file_name2) as f:
    for json_line in f:
        current_strs = ""
        prev_strs = ""
        # print(json_line)
        json_file = json.loads(json_line)
        current_conversation = json_file["dialog"]
        for i, dic in enumerate(current_conversation):
            if i == 0:
                current_strs = f"Speaker {i%2 + 1} : "+ dic["text"] + " "
            if i != 0 and i != len(current_conversation)-1:
                train_lists.append(current_strs)
                current_strs += f"Speaker {i%2 + 1} : "+ dic["text"] + " "
                label_lists.append(dic["text"])
            if i == len(current_conversation)-1:
                train_lists.append(current_strs)
                label_lists.append(dic["text"])

        prev_dialogs = json_file['previous_dialogs'][0]['dialog']
        for i, dic in enumerate(prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])

file_name3 = "bart_all_session_no_persona/session_3/train.jsonl"
with open(file_name3) as f:
    for json_line in f:
        current_strs = ""
        prev_strs = ""
        # print(json_line)
        json_file = json.loads(json_line)
        current_conversation = json_file["dialog"]

        for i, dic in enumerate(current_conversation):
            if i == 0:
                current_strs = f"Speaker {i%2 + 1} : "+ dic["text"] + " "
            if i != 0 and i != len(current_conversation)-1:
                train_lists.append(current_strs)
                current_strs += f"Speaker {i%2 + 1} : "+ dic["text"] + " "
                label_lists.append(dic["text"])
            if i == len(current_conversation)-1:
                train_lists.append(current_strs)
                label_lists.append(dic["text"])

        prev_prev_dialogs = json_file['previous_dialogs'][0]['dialog']
        for i, dic in enumerate(prev_prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])

        prev_dialogs = json_file['previous_dialogs'][1]['dialog']
        for i, dic in enumerate(prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])

file_name4 = "bart_all_session_no_persona/session_4/train.jsonl"
with open(file_name4) as f:
    for json_line in f:
        current_strs = ""
        prev_strs = ""
        # print(json_line)
        json_file = json.loads(json_line)
        current_conversation = json_file["dialog"]

        for i, dic in enumerate(current_conversation):
            if i == 0:
                current_strs = f"Speaker {i%2 + 1} : "+ dic["text"] + " "
            if i != 0 and i != len(current_conversation)-1:
                train_lists.append(current_strs)
                current_strs += f"Speaker {i%2 + 1} : "+ dic["text"] + " "
                label_lists.append(dic["text"])
            if i == len(current_conversation)-1:
                train_lists.append(current_strs)
                label_lists.append(dic["text"])

        prev_prev_prev_dialogs = json_file['previous_dialogs'][0]['dialog']
        for i, dic in enumerate(prev_prev_prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])

        prev_prev_dialogs = json_file['previous_dialogs'][1]['dialog']
        for i, dic in enumerate(prev_prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])

        prev_dialogs = json_file['previous_dialogs'][2]['dialog']
        for i, dic in enumerate(prev_dialogs):
            if i == 0:
                prev_strs = f"Speaker {i%2 + 1} : "+ dic['text'] + " "
            if i != 0 and i != len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                prev_strs += f"Speaker {i%2 + 1} : "+ dic['text'] + " "
                label_lists.append(dic['text'])
            if i == len(prev_dialogs)-1:
                train_lists.append(prev_strs)
                label_lists.append(dic["text"])
lengdata = len(train_lists)
print(lengdata)

286257


In [6]:
valid_lists = train_lists[int(lengdata*0.7):int(lengdata*0.9)]
valid_label_lists = label_lists[int(lengdata*0.7):int(lengdata*0.9)]

test_lists = train_lists[int(lengdata*0.9):]
test_label_lists = label_lists[int(lengdata*0.9):]

train_lists = train_lists[:int(lengdata*0.7)]
train_label_lists = label_lists[:int(lengdata*0.7)]
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 512


In [20]:
encodings = tokenizer("\n\n".join(valid_lists), return_tensors='pt')

In [ ]:
# import math
# def score(sentence):
#     tokenize_input = tokenizer.tokenize(sentence)
#     tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
#     loss = model(tensor_input, labels = tensor_input)
#     return math.exp(loss.loss.item())

# # print([score(i) for i in test_lists])
# for i in test_lists:
#     print(score(i))
#     break

In [41]:
import math
import statistics


model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
max_length = model.config.n_positions
stride = 512
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        
        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

# ppl = math.exp(torch.stack(nlls).mean()) * 10
ppl = math.exp(torch.stack(nlls).mean())

100%|█████████▉| 18114/18116 [12:28<00:00, 24.21it/s]


In [46]:
print("Perplexity : ", ppl)

Perplexity :  17.73265055126779


In [ ]:
from transformers import set_seed
from rouge import Rouge
from evaluate import load
perplexity = load("perplexity", module_type="metric")
results = 0
rouge = Rouge()
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

filename = "./output_msc_2_gpt_no_persona.txt"
total_f1 = 0
pred_arr = []
ref_arr = []

for i in range(len(test_lists)):
    model_inputs = tokenizer(test_lists[i], return_tensors='pt')

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    sample_outputs = model.generate(
        **model_inputs,
        max_new_tokens=40,
        do_sample=True,
        top_k=50,
        top_p=0.95, 
    ) 
    generation_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    pred_arr.append(generation_text[len(test_lists[i])-1:])
    with open(filename, "a") as f:
        f.write(f"\nprediction: \n{generation_text[len(test_lists[i])-1:]}\ngt     : \n{test_label_lists[i]}")
    total_f1 += rouge.get_scores(generation_text[len(test_lists[i])-1:], test_label_lists[i], avg=True)['rouge-1']['f']


In [ ]:
'''rouge랑 perplexity를 다시 구하는 코드'''
total_f1 /= len(test_lists)
print("Rouge : ", total_f1)
results = perplexity.compute(predictions=pred_arr, model_id='gpt2')
print("Perplexity : ", round(results["mean_perplexity"], 2))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu = 0

for i in range(len(test_lists)):
    model_inputs = tokenizer(test_lists[i], return_tensors='pt')

    # set top_k = 50 and set top_p = 0.95
    sample_outputs = model.generate(
        **model_inputs,
        max_new_tokens=40,
        do_sample=True,
        top_k=50,
        top_p=0.95, 
    ) 
    generation_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    bleu += sentence_bleu(test_label_lists[i], generation_text[len(test_lists[i])-1:], weights=(1,0,0,0))



In [ ]:
bleu /= len(test_lists)
print("BLEU : ", bleu)

In [7]:
!pip install git

'git'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [1]:
import pytreebank
# load the sentiment treebank corpus in the parenthesis format,
# e.g. "(4 (2 very ) (3 good))"
dataset = pytreebank.load_sst()
# add Javascript and CSS to the Ipython notebook
pytreebank.LabeledTree.inject_visualization_javascript()
# select and example to visualize
example = dataset["train"][0]
# display it in the page
example.display()

ModuleNotFoundError: No module named 'pytreebank'